In [1]:
import tensorflow as tf
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate
import os
import numpy as np
from PIL import Image
from keras.layers import Cropping2D


In [2]:
# Load the images
image_dir = "../Images/"
images = os.listdir(image_dir)
images = [os.path.join(image_dir, image) for image in images]

# Load the masks
mask_dir = "../Masks/"
masks = os.listdir(mask_dir)
masks = [os.path.join(mask_dir, mask) for mask in masks]

# Preprocess the images and masks
img_shape = (256, 256, 1)
images_arr = np.zeros((len(images),) + img_shape, dtype=np.float32)
masks_arr = np.zeros((len(masks),) + img_shape, dtype=np.float32)

for i, img_path in enumerate(images):
    img = Image.open(img_path).convert('L').resize((img_shape[0], img_shape[1]))
    images_arr[i] = np.array(img).reshape(img_shape)

for i, mask_path in enumerate(masks):
    mask = Image.open(mask_path).convert('L').resize((img_shape[0], img_shape[1]))
    masks_arr[i] = np.array(mask).reshape(img_shape)

# Return the images and masks
print(images_arr.shape, masks_arr.shape)

(601, 256, 256, 1) (601, 256, 256, 1)


In [3]:
# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
train_images, test_images, train_masks, test_masks = train_test_split(images_arr, masks_arr, test_size=0.2, random_state=42)

# Print the shapes of the training and testing sets
print("Training set:", train_images.shape, train_masks.shape)
print("Testing set:", test_images.shape, test_masks.shape)

Training set: (480, 256, 256, 1) (480, 256, 256, 1)
Testing set: (121, 256, 256, 1) (121, 256, 256, 1)


In [5]:
def unet_updated(input_shape):
    inputs = Input(input_shape)
    
    # Encoder Block 1
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    # Encoder Block 2
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Encoder Block 3
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    # Encoder Block 4
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    
    # Bottom Block
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)
    
    # Decoder Block 1
    up6 = UpSampling2D(size=(2, 2))(drop5)
    up6 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(up6)
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

        # Decoder Block 2
    up7 = UpSampling2D(size=(2, 2))(conv6)
    up7 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(up7)
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

    # Decoder Block 3
    up8 = UpSampling2D(size=(2, 2))(conv7)
    up8 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(up8)
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

    # Decoder Block 4
    up9 = UpSampling2D(size=(2, 2))(conv8)
    up9 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(up9)
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    
    # Output Block
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)
    
    # Instantiate the model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    return model



In [6]:
# unet((227, 227, 1)).summary()
model = unet_updated((256, 256, 1))

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 113, 113, 128), (None, 112, 112, 128)]

In [ ]:
# Build the model
model = unet_updated((256, 256, 1))



In [ ]:
# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])



In [ ]:
# Define the EarlyStopping callback
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)

In [ ]:
# Train the model
model.fit(train_images, train_masks, epochs=50)



Epoch 1/50
3/3 [==============================] - 137s 34s/step - loss: -7132707840.0000 - accuracy: 0.3410
Epoch 2/50


In [ ]:
# Evaluate the model
model.evaluate(test_images, test_masks)



In [ ]:
# Save the model
model.save("unet.h5")
